In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

2024-10-15 13:13:05.386547: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-15 13:13:05.394306: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-15 13:13:05.415651: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 13:13:05.447164: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 13:13:05.457441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 13:13:05.479140: I tensorflow/core/platform/cpu_feature_gu

In [2]:
# 1. Load the preprocessed data
data_path = '/home/moraa/Documents/Machine-learning/Gas_Sensor/artifacts/preprocessed/preprocessed_data.csv'
data = pd.read_csv(data_path)


In [3]:

X = data.drop(['gas_class', 'drift'], axis=1)  # Features
y_gas_class = data['gas_class']  # Target for gas classification
y_drift = data['drift']  # Target for drift prediction

# 2. Split the data into train and validation sets
X_train, X_val, y_train_gas_class, y_val_gas_class, y_train_drift, y_val_drift = train_test_split(
    X, y_gas_class, y_drift, test_size=0.2, random_state=42
)

KeyError: "['gas_class', 'drift'] not found in axis"

In [4]:
# 3. Load the prepared model
model_path = '/home/moraa/Documents/Machine-learning/Gas_Sensor/artifacts/prepared_model/model.keras'
model = tf.keras.models.load_model(model_path)

# 4. Compile the model (if not already compiled)
model.compile(
    optimizer='adam',
    loss={'gas_class_output': 'sparse_categorical_crossentropy', 'drift_output': 'mse'},
    metrics={'gas_class_output': 'accuracy', 'drift_output': 'mae'}
)


/home/moraa/Documents/Machine-learning/Gas_Sensor/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# 5. Train the model
history = model.fit(
    X_train,
    {'gas_class_output': y_train_gas_class, 'drift_output': y_train_drift},
    validation_data=(X_val, {'gas_class_output': y_val_gas_class, 'drift_output': y_val_drift}),
    epochs=50,
    batch_size=32
)
